# Classificação de litofácies através do algoritmo Support Vector Machine (SVM)

O processo de previsão das propriedades espaciais, principalmente da litologia (litofácies), é decisivo para o sucesso da produção de hidrocarbonetos em reservatórios de petróleo. Essa previsão pode ser guiada pela associação de dados físicos e de sísmica obtidos dos poços. 

Nesse nosso caso, os recursos serão dados de registro de poço de oito poços de gás. Esses poços já tiveram classes de litofácies atribuídas com base nas descrições de núcleos. Depois de treinar um classificador, vamos usá-lo para atribuir fácies a poços que não foram descritos.

Esse notebook aplicará a técnicas de Machine Learning denominada Support Vector Machine (SVM), para a classificação automática de litofácies de perfis de poço com base nos dados fornecidos no Geophysical Tutorial Machine Learning Contest 2016 (https://github.com/seg/2016-ml-contest).  

Para mais detalhes sobre SVM: https://en.wikipedia.org/wiki/Support-vector_machine.

# 1 - Importar as bibliotecas básicas

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 2 - Carregar o conjunto de dados

O conjunto de dados consiste em onze entradas: um rótulo de fácies em intervalos de profundidade de meio pé (numérico), seis medições de registros wireline (numérico), formação perfurada e nome dos poços (categóricos), profundidade (numérico) e um indicador da localização dos poços (numérico), com as seguintes legendas: 
 
- Facies: classes de litofácies atribuídas com base na análise prévia de testemunhos. 
- Formation: tipo de formação associada a cada medição.
- Well Name: o nome do poço no qual foram realizadas as medições.
- Depth: a profundidade na qual foram realizadas as medições.
- GR (Gamma-Ray): Perfil de raios gama.
- ILD_log10: Perfil de resistividade.
- DeltaPHI: Diferença de porosidade de densidade de nêutrons
- PHIND: Porosidade média de densidade de nêutrons.
- PE: Perfil de efeito fotoelétrico.
- NM_M: Indicador não marinho (1)/ marinho (2)
- RELPOS: Posição relativa

In [ ]:
df = pd.read_csv('../input/datasetmlcontest2016/data.csv')
print(df.shape)
df

Podemos ver na linha de contagem que temos um total de 3232 amostras para cada uma das 11 entradas do conjunto de dados.

Existem nove classes de fácies (numeradas de 1 a 9) identificadas no conjunto de dados. 

In [ ]:
np.sort(df.Facies.unique())

Vamos verificar a distribuição de classes para os dados marinhos e não marinhos.

In [ ]:
np.sort(df.loc[df['NM_M']==1].Facies.unique())

In [ ]:
np.sort(df.loc[df['NM_M']==2].Facies.unique())

Vemos que a classe 9 não está presente nos dados não marinhos e a classe 1 não está presente nos dados marinhos.

O conjunto de dados possue ao todo 8 poços.

In [ ]:
df['Well Name'].unique()

E os poços cruzam um total de 14 formações diferentes.

In [ ]:
df['Formation'].unique()

O comando abaixo mostra a descrição estatística geral do conjunto de dados.

In [ ]:
df.describe()

É importante verificar se não há dados ausentes no conjunto de dados. Na linha 'count' podemos observar que todos os recursos possuem 3232 medições e nas medidas estatísticas temos valores numéricos, o que indica que não há dados ausentes. 

Podemos confirmar isso através do comando abaixo, que nos dará a quantidade de dados ausentes nas colunas do conjunto de dados. 

In [ ]:
df.isnull().sum()

# 3 - Visualizar dos dados

Os gráficos cruzados são uma ótima ferramenta para visualizar como duas propriedades variam com o tipo de rocha.

In [ ]:
df_plot = df.drop('NM_M', axis=1)
g = sns.pairplot(df_plot, hue = 'Facies', diag_kind = 'kde',  kind='scatter', height=2.5,
            plot_kws = {'alpha': 0.6, 's': 80, 'edgecolor': 'k'}, palette='Spectral')

sns.set_context("paper", font_scale=2.0)
    
plt.show()

Vemos que as classes estão longe de serem linearmente separáveis e por isso a escolha do algoritmo SVM para o processo de classificação. De modo resumido, o que ele faz é transportar os recursos para um dimensão superior, chamada dimensão de recursos, onde espera-se que estes sejam linearmente separáveis.

Vamos checar a distribuição das frequencias de cada classe dentro do banco de dados.

In [ ]:
df.Facies.value_counts(sort=True).plot(kind='bar')
plt.ylabel('Frequencia')
plt.xlabel('Litofacies')
plt.show()

Pode-se notar que há um certo grau de desbalanço entre as litofácies, que pode ser visualizado em termos percentuais pelo comando abaixo.  

In [ ]:
round((df['Facies'].value_counts(sort=True))*100/len(df['Facies']),2)

As classes 2,3,8 e 6 tendem a serem aprendidas com maior facilidade, pelas suas frequências e o relativo equilíbrio entre elas. Espera-se que o modelo tenha maiores dificuldades no aprendizado das classes 1,5,4,9 e 7, pelo motivo oposto.

O desbalanço entre as classes também pode induzir o modelo a um invés no momento das predições. 

# 4 - Contrução do modelo

In [ ]:
from sklearn.svm import SVC # importa o modelo SVM
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

Vamos inicialmente separar os dados em treinamento e teste. Vamos escolher como poço de teste o 'SHANKLE', ao passo que o modelo será treinado nos demais poços. 

In [ ]:
test_well = df[df['Well Name'] == 'SHANKLE']
df_train = df[df['Well Name'] != 'SHANKLE']

In [ ]:
df_train.Facies.value_counts(sort=True).plot(kind='bar')
plt.ylabel('Frequencia')
plt.xlabel('Litofacies')
plt.show()

In [ ]:
test_well.Facies.value_counts(sort=True).plot(kind='bar')
plt.ylabel('Frequencia')
plt.xlabel('Litofacies')
plt.show()

Para evitar problemas com o "vazamento de dados" (data leakage), vamos criar um pipeline para preparação dos dados dentro do processo de avaliação e validação do modelo nos dados de treinamento.

- Para mais detalhes ver: https://en.wikipedia.org/wiki/Leakage_(machine_learning)

O modelo SVM tem alguns parâmetros para serem ajustados, isso será feito através da função GridSearchCV por meio de uma validação cruzada com k-fold=5 e métrica f1-score.
 
- A função GridSearchCV não é o modo mais otimizado de se fazer ajuste de parâmetros. No entanto, como esse exemplo tem pequena dimensão, optamos por seu uso.  
 

Para mais detalhes:

- https://scikit-learn.org/stable/modules/svm.html
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
- https://scikit-learn.org/stable/modules/grid_search.html

O modelo será construido com base nos recursos: Depth, GR, ILD_log10, DeltaPHI, PHIND, PE, NM_M e RELPOS.

In [ ]:
def df_Xy_Split(df):
    X = df.drop(['Facies', 'Formation', 'Well Name'], axis=1)
    y = df['Facies']
    
    return X,y

In [ ]:
X_train,y_train = df_Xy_Split(df_train)

In [ ]:
X_train

In [ ]:
# definir o pipeline
steps = list()
steps.append(('scaler', StandardScaler()))
steps.append(('svc', SVC()))
pipeline = Pipeline(steps=steps)

# definir os procedimentos de ajuste de parâmetros
cv = RepeatedKFold(n_splits=5, n_repeats=10, random_state=0)

svm_parameters = {'svc__C':  np.linspace(30,100,20), 
                   'svc__gamma':  [0.0001, 0.0005, 0.001, 0.005]}

svm_grid = GridSearchCV(pipeline, svm_parameters, scoring='f1_macro', n_jobs =-1, cv=cv).fit(X_train,y_train)
svm_clf = svm_grid.best_estimator_.fit(X_train,y_train)

print('*' * 80)
print('Pontuação média:\t\t{:.2%}'.format(np.mean(svm_grid.cv_results_['mean_test_score'])))
print('Melhor pontuação:\t\t{:.2%}'.format(svm_grid.best_score_))
print('Parametros:\t{}'.format(svm_grid.best_params_))
print('*' * 80)

plt.hist(svm_grid.cv_results_['mean_test_score'])
plt.title('Distribuição do f1-score na validação cruzada')
plt.xlabel('F1-Score')
plt.ylabel('Frequencia')
plt.show()

- Pelo gráfico acima nota-se que a maioria das pontuações F1 estão entre 42 e 60%, aproximadamente.
 
- Outro ponto a se destacar é que a pontuação média e máxima estão relativamente próximas também. 
 
- As pontuações mais baixas podem ser explicadas pelas complexas variações litológicas entre os poços que compõem o conjunto de dados de treinamento, o que pode impor muitas dificuldades ao algoritmo em algumas dobras da validação cruzada. 
 
- Nesse ponto, pode-se testar outros métodos para se criar as dobras da validação cruzada e analisar o comportamento do algoritmo para cada uma delas.  
 
- Para mais detalhes: https://scikit-learn.org/stable/modules/cross_validation.html.


# 5 - Predições do modelo

In [ ]:
from matplotlib.pyplot import cm

Aplicaremos abaixo o mesmo Pipeline utilizado no treinamento nos dados de teste. 

In [ ]:
# Separar recursos e alvos
X_test,y_test = df_Xy_Split(test_well)

In [ ]:
# Previsões do modelo
svm_pred = svm_clf.predict(X_test)

# 6 - Avaliação do modelo

In [ ]:
from sklearn.metrics import plot_confusion_matrix
from sklearn import metrics

Para avaliação do modelo usaremos as métricas clássicas: acurácia, precisão, recall e f1-score.

- A precisão e recall são métricas que nos dizem como o classificador está se saindo para fácies individuais. 

- A precisão é a probabilidade de que, dado um resultado de classificação para uma amostra, a amostra realmente pertença a essa classe.

- Recall é a probabilidade de que uma amostra seja classificada corretamente para uma determinada classe. 

- A pontuação F1 combina o recall e a precisão para fornecer uma única medida de relevância dos resultados do classificador.

- Para mais detalhes sobre métricas de classificação: https://bit.ly/genesisMetrics1

In [ ]:
# Métricas do modelo
print(metrics.classification_report(y_test, svm_pred))

In [ ]:
# Matriz de Confusão
target_names = np.unique(y_test).tolist()
f, ax = plt.subplots(figsize=(8, 6))
mat = metrics.confusion_matrix(y_test, svm_pred)
g = sns.heatmap(mat.T, annot=True, fmt="d",  cmap='viridis', cbar=False, linewidths=0.0, annot_kws={"size": 12}, 
                xticklabels=target_names ,  yticklabels=target_names)
ax.set_title('Matriz de Confusão', fontsize = 16)
ax.set_ylabel('Litofácies Reais', fontsize = 14) 
ax.set_xlabel('Litofácies Preditas', fontsize = 14)
plt.show()

- A acurácia total do modelo foi 49%, ao passo que as médias de precisão, recall e f1-score foram de 58, 44 e 44%, respectivamente. 
- Nota-se também que o modelo teve melhor desempenho na classificação das classes mais frequentes e pior nas menos frequentes, como esperado. 
- Esses resultados evidenciam a complexidade do problema de classificação de litofácies, devido às relações fortemente não lineares entre as medidas de perfis e as variações litológicas. 

Essa abordagem supera a descrita em Hall, B. (2016), o qual também usa SVM no mesmo problema.

- Hall, B. (2016). Facies classification using machine learning. The Leading Edge, 35(10), 906–909. doi:10.1190/tle35100906.1 

Vamos construir um gráfico para ilustrar a distribuição da litofacies reais e preditas ao longo do poço, em relação aos recursos do modelo.

In [ ]:
import matplotlib.colors as colors

In [ ]:
# Criar um dicionário de cores para as litofácies
facies_colors = ['#F4D03F', '#F5B041','#DC7633','#6E2C00','#1B4F72','#2E86C1', '#AED6F1', '#A569BD', '#196F3D']
facies_labels = np.sort(df.Facies.unique())

facies_color_map = {}
facies_number_dic = {}
for ind, label in enumerate(facies_labels):
    facies_color_map[label] = facies_colors[ind]
    facies_number_dic[label] = ind+1 

In [ ]:
def data_plot(X,y, y_pred, facies_colors = facies_colors):

    df = X.sort_values(by='Depth')
    
    ztop=df.Depth.min(); zbot=df.Depth.max()
    ncols = len(df.columns)+1
    f, ax = plt.subplots(nrows=1, ncols=ncols, figsize=(18, 12))
    
    color=iter(cm.rainbow(np.linspace(0,1,ncols)))
    
    for i in np.arange(1,ncols-1):
        col_name = df.iloc[:,i].name
        c = next(color)
        ax[i-1].plot(df.iloc[:,i], df.Depth, c=c)
        ax[i-1].set_xlabel(col_name, fontsize = 14); 
 
        ax[i].set_yticklabels([]);
        
        ax[i-1].set_ylim(ztop,zbot)
        ax[i-1].invert_yaxis()
        ax[i-1].grid([])
        ax[i-1].locator_params(axis='x', nbins=3)
    
    ax[0].set_ylabel("Profundidade (ft)", fontsize = 20)
    
    ax[-1].grid([]); ax[-1].set_xlabel('Predição', fontsize = 14); 
    ax[-1].set_yticklabels([]); ax[-1].set_xticklabels([]);
    
    ax[-2].grid([]); ax[-2].set_xlabel('Real', fontsize = 14); 
    ax[-2].set_yticklabels([]); ax[-2].set_xticklabels([]);    
    
    # plotar as cores das petrofacies 
    cmap_faces = colors.ListedColormap(facies_colors[0:len(facies_colors)], 'indexed')
    cluster1 = np.repeat(np.expand_dims(y.values,1), 100, 1)
    im1 = ax[-2].imshow(cluster1, interpolation='none', aspect='auto', cmap=cmap_faces,vmin=1,vmax=9)
    cluster2 = np.repeat(np.expand_dims(y_pred,1), 100, 1)
    im2 = ax[-1].imshow(cluster2, interpolation='none', aspect='auto', cmap=cmap_faces,vmin=1,vmax=9)


In [ ]:
data_plot(X_test,y_test, svm_pred)